<a href="https://colab.research.google.com/github/davidcrispell/LLM-PC/blob/main/TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Setup

In [ ]:
    %pip install transformer_lens
    %pip install einops
    %pip install jaxtyping

In [18]:
import sys
import torch as t
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
import numpy as np
import einops
from jaxtyping import Int, Float
import functools
from tqdm import tqdm
from IPython.display import display
from transformer_lens.hook_points import HookPoint
from transformer_lens import (
    utils,
    HookedTransformer,
    HookedTransformerConfig,
    FactoredMatrix,
    ActivationCache,
)
from transformers import AutoTokenizer, AutoModel, LlamaForCausalLM
device = t.device("cuda" if t.cuda.is_available() else "cpu")

#### Load model using TransformerLens

In [35]:
LLAMA_PATH = "LLM-PBE/Llama3.1-8b-instruct-LLMPC-Red-Team"

tokenizer = AutoTokenizer.from_pretrained(LLAMA_PATH)
hf_model = LlamaForCausalLM.from_pretrained(LLAMA_PATH, low_cpu_mem_usage=True)

# model = HookedTransformer.from_pretrained(LLAMA_PATH, device="cpu", fold_ln=False, center_writing_weights=False, center_unembed=False)

hf_model = hf_model.to("cuda" if t.cuda.is_available() else "cpu")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [28]:
# hf_model = hf_model.to("cuda" if t.cuda.is_available() else "cpu")
hf_model.generate(t.tensor([tokenizer("The capital of Germany is")["input_ids"]]), max_new_tokens=20, temperature=0.1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tensor([[128000,    791,   6864,    315,  10057,    374,  20437,     13,   1102,
            374,    264,  34076,   3363,    449,    264,   9257,   3925,    323,
           7829,     13,  20437,    374,   3967,    369,   1202,   6366]])

In [34]:
tokenizer.decode([128000,    791,   6864,    315,  10057,    374,  20437,     13,   1102,
            374,    264,  34076,   3363,    449,    264,   9257,   3925,    323,
           7829,     13,  20437,    374,   3967,    369,   1202,   6366]

)

'<|begin_of_text|>The capital of Germany is Berlin. It is a vibrant city with a rich history and culture. Berlin is known for its beautiful'